In [1]:
import spacy
import pandas as pd
import re
import os
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel
import pyLDAvis.gensim_models
import warnings
from tqdm import tqdm
from mistralai import Mistral
import time

In [2]:
nlp = spacy.load("fr_core_news_lg")

In [5]:
my_stop = ["janvier", "février", "fevrier", "mars", "avril", "mai", "juin", "juillet", "août", "aout", "septembre", "octobre", "novembre", "décembre",
           "je", "tu", "il", "elle", "nous", "vous", "ils", "elles", "ça", "c'", "ce", "cela", "ceci", "moi", "toi", "lui", "sien", "leur", 
          "faire", "être", "avoir", "mettre", "aller", "devoir", "vouloir", "falloir", "devenir", "luire", "pouvoir", "savoir", "demeurer", "sembler",
           "passer", "voir", "venir", "prendre", "trouver", "raconter", "déclarer",
           "expliquer", "dire", "annoncer", "confirmer", 
          "sont", "est", "a", "ont", "vont", "seraient", "seront",
          "jusque", "mais", "et", "donc", "or", "ni", "car", "jusqu", "jusqu'", "bien", "mal", "presque", "jusqu’",
          "an", "année", "jour", "mois",
          "...", "non", "oui", "presscontact",
          "lundi", "mardi", "mercredi", "jeudi", "vendredi", "samedi", "dimanche", "hier", "demain", "aujourd'hui",
          "abonné", "m.", "mme", "ici", "-t", "là", "là-bas", "-là", "cas", "trop",
          "premier", "deuxième", "troisième", "dernier", "prochain",
          "toujours", "jamais", "parfois",
          "abonné", "article", "réservé", "e.a.", "s.t."
          "grand", "petit", "gros", "large",
          "afp", "belga", "the", "rien", "conseiller", "rédaction", "figaro",
          "femme", "homme",
           "évident", "judicieux"
          #"prostitué", "prostituée", "prostitution", "prostituer",
          'antérieur', 'toi', 'lors', 'il', 'moindres', 'pres', 'chaque', 'avaient', 'vôtre', 'parfois', 'siens', 'quel', 'nos', 'restant', 'restent', 'toi-même', "c'", 'jusque', 'la', 'm’', 'lesquels', 'anterieures', 'pourrais', 'chez', 'en', 'vous-mêmes', 'huit', 'envers', 'celles-là', 'tout', 'suis', 'douzième', 'certes', 'apres', 'moins', 'cinquième', 'était', 'ouvert', 'etaient', 'reste', 'directe', 'nôtres', 'leurs', 'on', 'd’', 'doivent', 'anterieure', 'différents', 'hi', 'deux', 'etant', 'lès', 'avait', 'ha', 'dont', 'être', 'ma', 'dix-neuf', 'elle', 'toute', 'auquel', 'sept', 'desquelles', 'etc', 'ça', 'dans', 'alors', 'sera', 'parmi', 'nous-mêmes', 'plutôt', 'vous', 'auront', 'stop', 'certain', 'seules', 'basee', 'préalable', 'devant', 'anterieur', 'allaient', 'c’', 'quelconque', 'onze', 'excepté', 'font', 'façon', 'étais', 'le', 'delà', 'lesquelles', 'ceux', 'neuvième', 'vôtres', 'auxquels', 'où', 'votres', 'prealable', 'toujours', 'eux', 'malgre', 'doit', 'toi-meme', 'siennes', 'y', "n'", 'ait', 's’', 'ne', 'quatre', 'cinquantaine', 'pendant', 'vé', 'elles', 'celui-la', 'té', 'qui', 'ayant', 'aurait', 'avoir', 'cent', 'ce', 'nombreux', 'suit', 'seuls', 'quelle', 'sent', 'voilà', 'votre', 'spécifique', 'différent', 'tels', 'aupres', 'seule', 'fait', 'attendu', 'huitième', 'vos', 'dire', 'depuis', 'nouveau', 'soi-meme', 'suivant', 'à', 'seul', 'suffit', 'des', 'rend', 'pu', 'pas', 'comme', 'tant', 'eh', 'j’', 'moi-même', "qu'", 'deuxièmement', 'ai', 'eu', 'tente', 'i', 'ceux-ci', 'ta', 'également', 'étant', 'celles-la', 'chacun', "s'", 'vas', 'fais', 'enfin', 'leur', 'suivre', 'auxquelles', 'trente', 'quelles', 'ouverts', 'seraient', 'suivants', 'tienne', 'relative', 'celle', 'toutes', 'environ', 'meme', 'malgré', 'moi', 'souvent', 'merci', 'six', 'differente', 'quatre-vingt', 'importe', 'specifique', 'aux', 'tend', 'est', 'possible', 'celles', 'elle-meme', 'ceux-là', 'désormais', 'mien', 'trois', 'soixante', 'quatrièmement', 'seize', 'neanmoins', 'vu', 'da', 'autres', 'quinze', 'pouvait', 'egalement', 'n’', 'puis', 'dedans', 'surtout', 'une', 'divers', 'là', 'tiens', 'puisque', 'néanmoins', 'dix-huit', 'dehors', 'effet', 'lorsque', 'vais', 'peut', 'tu', 'comment', 'les', 'lui-meme', 'premier', 'selon', 'combien', 'cela', 'faisaient', 'revoilà', 'hui', 'déja', 'sans', 'certaines', 'tous', 'concernant', 'lequel', 'certains', 'te', 'un', 'je', 'etais', 'va', 'avant', 'quelque', 'hep', 'l’', 'compris', 'proche', 'pense', 'si', 'plusieurs', 'du', 'différentes', 'me', 'au', 'cinquante', 'autrement', 'plus', 'durant', 'onzième', 'cet', 'cinq', 'car', 'memes', 'mon', 'nombreuses', 'parle', 'sous', 'première', 'soi', 'semblable', 'ces', 'sien', 'vers', 'hem', 'derrière', 'donc', 'ni', 'sienne', 'debout', 'ainsi', 'celle-ci', 'eux-mêmes', 'semblaient', 'es', 'soi-même', 'quelques', 'procedant', 'tellement', 'telles', 'mes', "d'", 'pour', 'tenir', 'treize', 'plutot', 'seront', 'cinquantième', 'certaine', 'faisant', 'elles-memes', 'celui-ci', 'partant', 'uns', 'celles-ci', 'antérieure', 'suivante', "t'", 'laisser', 'dit', 'mienne', 'ouias', 'chacune', 'ès', 'dits', 'unes', 'ci', 'derriere', 'hou', 'retour', 'spécifiques', 'laquelle', 'quand', 'tes', 'tres', 'houp', 'directement', 'voila', 'mille', 'miennes', 'septième', 'bat', 'différente', 'assez', 'autrui', 'cependant', 'antérieures', 'pourrait', 'elles-mêmes', 'devra', 'quiconque', 'voici', 'feront', 'douze', 'exactement', 'aussi', 'ceci', 'gens', 'et', 'longtemps', 'ses', 'etre', 'different', 'hé', 'aura', 'dessus', 'devers', 'quatorze', "j'", 'allons', 'soit', 'aie', "l'", 'hormis', 'premièrement', 'rendre', 'vingt', 'sait', 'quatrième', 'diverse', 'tel', 'dite', 'touchant', 'étaient', 'ont', 'ouste', 'desormais', 'quant-à-soi', 'semble', 'nous', 'peu', 'via', 'diverses', 'dessous', 'se', 'mêmes', 'sinon', 'telle', 'dix-sept', 'quoi', 'son', 'troisièmement', 'semblent', 'nul', 'déjà', 'bas', 'hue', 'peuvent', 'lui-même', 'qu’', 'differentes', 'mais', 'celle-la', 'ils', 'â', 'quarante', 'abord', 'près', 'a', 'pourquoi', 'seulement', 'moi-meme', 'celui-là', 'ouverte', 'desquels', 'dixième', 'dix', 'sur', 'auraient', 'duquel', 'même', 'nôtre', 'suffisante', 'dejà', 'dès', 'relativement', 'avais', 't’', 'deja', 'maint', 'personne', 'peux', 'ho', 'sixième', 'specifiques', 'suivantes', 'que', 'avons', 'entre', 'o', 'tiennes', 'ton', 'differents', 'elle-même', 'as', 'permet', 'parlent', 'precisement', 'outre', 'lui', 'maintenant', 'quels', "m'", 'ah', 'serait', 'autre', 'quant', 'tien', 'cette', 'sa', 'précisement', 'celle-là', 'vont', 'possibles', 'afin', 'troisième', 'miens', 'encore', 'ou', 'sont', "quelqu'un", 'etait', 'juste', 'deuxième', 'de', 'parler', 'sauf', 'revoila', 'na', 'notamment', 'or', 'très', 'facon', 'ô', 'avec', 'notre', 'revoici', 'par', 'après', 'tenant', 'jusqu', 'parce', 'hors', 'suffisant', 'quoique', 'celui']

verbes_d_etat = ['accorder', 'affecter', 'afficher', 'allouer', 'apparaître', 'appartenir', 'assigner', 'attester', 'attribuer', 'avoir', 'choisir', 'communiquer', 'comprendre', 'compter', 'concerner', 'concéder', 'confier', 'consacrer', 'consister', 'constituer', 'contenir', 'demeurer', 'destiner', 'devenir', 'diffuser', 'dispatcher', 'disposer', 'distribuer', 'décider', 'dédier', 'démontrer', 'désigner', 'déterminer', 'dévoiler', 'exister', 'exposer', 'exprimer', 'fixer', 'impliquer', 'imprimer', 'inclure', 'indiquer', 'installer', 'introduire', 'localiser', 'manifester', 'montrer', 'nommer', 'octroyer', 'opter', 'paraître', 'partager', 'placer', 'positionner', 'posséder', 'prouver', 'préférer', 'présenter', 'publier', 'regarder', 'remettre', 'renfermer', 'représenter', 'rester', 'répartir', 'réserver', 'révéler', 'sembler', 'signifier', 'situer', 'symboliser', 'sélectionner', 'toucher', 'transmettre', 'témoigner', 'valoir', 'vouer', 'éditer', 'élire', 'établir', 'être']

my_stop.extend(verbes_d_etat)

In [3]:
def clean_text(text):
    motifs = [r"\b(\d{1,2}h(?:\d{2})?)\b", #heures
              r"\b(\d{1,2} heures)\b", #heures toutes lettres
              r"\b(\d{2}/\d{2}/\d{2,4}|\d{2} (janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|décembre) \d{4})\b" #dates
              r"\d"
             ]
    for motif in motifs:
        text = re.sub(motif, '', text)
    return text

In [4]:
def import_txt(directory):
    txt_files = os.listdir(directory)
    articles = []
    journaux = []
    years = []
    months = []
    for txt in txt_files:
        try:
            with open(os.path.join(directory, txt), 'r', encoding='utf-8') as f:
                articles.append(clean_text(f.read()))
            cat, pol, auth, year, month, name = txt.split('_')
            journaux.append(auth)
            years.append(year)
            months.append(month)
        except OSError:
            print(colored(f"Erreur lors de l'ouverture/lecture du fichier {txt}.", 'red'))
    d = {'texte': articles, 'journal': journaux, 'année':years, 'mois':months}
    return pd.DataFrame(data = d, columns = ['texte', 'journal', 'année', 'mois'])

In [19]:
def tokenize_df(df, my_stop):
    txt = df['texte']
    docs = []
    for doc in tqdm(nlp.pipe(txt), total=len(txt)):
        tokens = []
        for token in doc:
            if token.orth_ > 2 and not token.lemma_.lower() in my_stop: # supprime les mots 
                tokens.append( token.lemma_.lower() )
        docs.append(tokens)
    return docs

In [7]:
def find_bigram(tokens):
    bigram = Phrases(tokens, min_count=5)

    for index in range(len(tokens)):
        for token in bigram[tokens[index]]:
            if '_' in token:  # les bigrammes peuvent être reconnus par "_" qui concatène les mots
                tokens[index].append(token)
    return tokens

In [8]:
def doc2bow(bigrams):
    dictionary = Dictionary(bigrams)
    print('Nombre de mots unique dans les documents initiaux :', len(dictionary))

    dictionary.filter_extremes(no_below=2, no_above=0.90)
    print('Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents :', len(dictionary))

    #print("Exemple :", dictionary.doc2bow(bigrams[4]))
    corpus = [ dictionary.doc2bow(doc) for doc in bigrams]
    return corpus, dictionary

In [9]:
def get_df(year:str):
    df = pd.DataFrame(columns = ['texte', 'journal', 'année', 'mois'])
    path = 'Sources/Europresse/txt/'
    for month in range(1, 13):
        month = str(month) if month > 9 else '0' + str(month)
        folder = path + month + '_' + year
        df_temp = import_txt(folder)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [10]:
def get_param(year:str, my_stop):
    print("Génération du dataframe")
    df = get_df(year)
    print("Tokenization du dataframe")
    tokens = tokenize_df(df, my_stop)
    print("Etablissement des bigrams")
    bigrams = find_bigram(tokens)
    print("Constitution du corpus et du dictionnaire")
    corpus, dictionary = doc2bow(bigrams)
    return df, tokens, bigrams, corpus, dictionary

In [11]:
def get_visu(corpus, dictionary, year:str):
    model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=14, chunksize=1000, passes=10, random_state=26112002)
    vis_data = pyLDAvis.gensim_models.prepare(model, corpus, dictionary, sort_topics=False)
    pyLDAvis.save_html(vis_data, 'html/LDA_' + year + '.html')
    return model        

In [28]:
def init_client():
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "mistral-large-latest"

    client = Mistral(api_key=api_key)
    return client

In [12]:
def get_topic_name(words:str, client):
    chat_response = client.chat.complete(
    model= "mistral-large-latest",
    messages = [
            {
            "role": "user",
            "content": f"""J'essaie de déterminer les sujets traités par des articles de journaux. Je vais te donner 20 mots qui sont représentatifs du sujet. Donne moi le thème abordé par les journaux en un ou deux groupes nominaux.

                            Les mots: {words}

                            Ne donne que le thème, n'ajoute rien d'autre""",
            },
        ]
    )
    
    return chat_response.choices[0].message.content

In [31]:
def find_topics(words_topic, client):
    topics = []
    for i, words in enumerate(words_topic):
        try:
            topics.append(get_topic_name(words, client))
            time.sleep(2)
        except SDKError:
            print(f"Erreur lors de la communication avec le LLM lors du traitement du {i}e sujet" ) 
    return topics   

In [30]:
def get_topics_full(model):
    words_topic = []
    for (topic, words) in model.print_topics(num_words=20):
        words = re.sub(r" \+ ", ', ', words)
        words = re.sub(r"\d.\d{3}\*", "", words)
        words_topic.append(words)
    client = init_client()
    topics = find_topics(words_topic, client)
    return topics

In [20]:
df20, tokens20, bigrams20, corpus20, dictionary20 = get_param('2020', my_stop)

Génération du dataframe
Tokenization du dataframe


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7800/7800 [11:01<00:00, 11.79it/s]


Etablissement des bigrams
Constitution du corpus et du dictionnaire
Nombre de mots unique dans les documents initiaux : 128738
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 70319


In [21]:
model20 = get_visu(corpus20, dictionary20, '2020')

In [32]:
topics20 = get_topics_full(model20)

In [33]:
topics20

['Programmation de films, cinéma',
 'Législation sur la prostitution, Politique de santé publique',
 'Mouvement féministe, Politique',
 'Conflits militaires, islamisme',
 'Industrie du divertissement, cinéma et musique',
 '\n\nProgrammestélévisés, séries et téléfilms',
 'Lutte contre la prostitution des jeunes',
 'Prostitution de mineurs, enquête judiciaire',
 'Littérature, histoire',
 'Musique et chanteurs célèbres',
 'Affaire criminelle, enquête policière',
 'Le procès de Jeffrey Epstein et Ghislaine Maxwell.',
 'Vie urbaine à Paris',
 'Cinéma et vie familiale']

In [34]:
df21, tokens21, bigrams21, corpus21, dictionary21 = get_param('2021', my_stop)

Génération du dataframe
Tokenization du dataframe


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7519/7519 [08:39<00:00, 14.48it/s]


Etablissement des bigrams
Constitution du corpus et du dictionnaire
Nombre de mots unique dans les documents initiaux : 135171
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 75009


In [35]:
model21 = get_visu(corpus21, dictionary21, '2021')

In [36]:
topics21 = get_topics_full(model21)

In [37]:
topics21

['Procès judiciaire, politique nationale',
 'Lutte contre la prostitution, aide sociale aux jeunes',
 'Violence sexuelle contre les mineurs',
 'Urbanisme et sécurité publique',
 'Affaire Jeffrey Epstein, Ghislaine Maxwell',
 'Prostitution et proxénétisme',
 'Littérature, arts de la scène',
 'Justice parlementaire à Bruxelles',
 'Cinéma, notamment le film "Le Silence des agneaux" et son actrice Jodie Foster.',
 'Militantisme féministe, droits sociaux',
 'Relations internationales en Asie',
 'Procès de Valérie Bacot, Affaire de justice',
 "L'industrie du cinéma et des séries télévisées",
 'Vie de famille']

In [38]:
df22, tokens22, bigrams22, corpus22, dictionary22 = get_param('2022', my_stop)

Génération du dataframe
Tokenization du dataframe


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7906/7906 [09:33<00:00, 13.78it/s]


Etablissement des bigrams
Constitution du corpus et du dictionnaire
Nombre de mots unique dans les documents initiaux : 147398
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 78496


In [39]:
model22 = get_visu(corpus22, dictionary22, '2022')

In [40]:
topics22 = get_topics_full(model22)

In [41]:
topics22

['Événements culturels, spectacles de théâtre',
 'Séries télévisées, diffusion',
 'Conflits internationaux, élections politiques',
 'Prostitution et proxénétisme, enquêtes judiciaires.',
 'Traite des êtres humains, Violence sexuelle sur mineurs',
 'Cinéma et festivals de films',
 'Cinéma',
 'procès judiciaire, système judiciaire français',
 'Variole du singe, vaccination',
 'Prostitution, violence sexuelle',
 'Exploitation sexuelle, accusations criminelles',
 'Littérature et histoire',
 'Arts et patrimoine culturel',
 'Problèmes sociaux en milieu urbain']

In [42]:
df23, tokens23, bigrams23, corpus23, dictionary23 = get_param('2023', my_stop)

Génération du dataframe
Tokenization du dataframe


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7046/7046 [09:46<00:00, 12.01it/s]


Etablissement des bigrams
Constitution du corpus et du dictionnaire
Nombre de mots unique dans les documents initiaux : 126524
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 67760


In [43]:
model23 = get_visu(corpus23, dictionary23, '2023')

In [44]:
topics23 = get_topics_full(model23)

In [45]:
topics23

['Johnny Hallyday, Laeticia Hallyday',
 'Actualités régionales',
 'Affaire politique américaine, Relations internationales États-Unis',
 'Industrie cinématographique et télévisuelle',
 'Littérature et arts',
 'Le trafic de drogue, les gangs criminels',
 'Procès judiciaire de Silvio Berlusconi',
 'Violences sexuelles',
 'Prostitution et proxénétisme',
 'Lutte contre la prostitution des jeunes de la rue',
 'Patrimoine urbain, événements historiques',
 'Famille, vie quotidienne',
 'Prostitution urbaine, politique municipale',
 'Arts et culture']

In [46]:
df24, tokens24, bigrams24, corpus24, dictionary24 = get_param('2024', my_stop)

Génération du dataframe
Tokenization du dataframe


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7106/7106 [08:37<00:00, 13.73it/s]


Etablissement des bigrams
Constitution du corpus et du dictionnaire
Nombre de mots unique dans les documents initiaux : 113005
Nombre de mots unique dans les documents après avoir enlevé les mots fréquents/peu fréquents : 63771


In [48]:
model24 = get_visu(corpus24, dictionary24, '2024')

In [49]:
topics24 = get_topics_full(model24)

In [50]:
topics24

['Procès pour agression sexuelle',
 'Vie urbaine',
 'Littérature, roman',
 'La prostitution des mineurs',
 'Trafic de drogue international, réseaux criminels.',
 'Programmes télévisés et cinéma.',
 'Arts et culture',
 'Violence criminelle et enquêtes judiciaires',
 'La Révolution française, la politique républicaine',
 'Violence sexuelle, Abus familial',
 'Service client, gestion des questions',
 'Enquête sur Donald Trump et Matt Gaetz',
 'Cinéma, festival de Cannes',
 'Prostitution de mineurs, réseaux de proxénétisme']

In [22]:
os.environ["MISTRAL_API_KEY"] = "wnH0JnTK4lSoh8Q2GHTe4PsmMrYeMjIW"

In [51]:
import pickle

In [63]:
with open("Sources/Europresse/pyData/tokens_LDA_2020.bin", "wb") as f:
    pickle.dump(tokens20, f)

In [64]:
with open("Sources/Europresse/pyData/tokens_LDA_2020.bin", "rb") as f:
    essai_toks = pickle.load(f)

In [66]:
essai_toks.orth_

AttributeError: 'list' object has no attribute 'orth_'